In [45]:
import re
import ast
import numpy as np
import matplotlib.pyplot as plt
import math
from matplotlib.lines import Line2D
import bisect as b
from scipy.stats import mannwhitneyu
from sklearn.metrics import auc
import itertools
import seaborn as sns
import pandas as pd
from tqdm import tnrange, notebook
import os
import pdb

In [46]:
def readData(filepath):
    with open(filepath, 'r') as file:
        data = file.read()
    data = np.array(ast.literal_eval(data))
    initial = data[0]
    return data[1]

In [47]:
def verticalCutAt(data, e):
    res = np.zeros([len(data)])
    for j,run in enumerate(data):
        run = np.array(run)
        index = b.bisect(run[:,0], e) - 1
        res[j] = run[index][1] if index >= 0 else float('-inf')
    return res

In [48]:
def getUniqueEvals(data):
    allEvals = []
    for arr in data:
        for e, _ in arr:
            allEvals.append(e)
    uniqueEvals = set(allEvals)
    return uniqueEvals

In [49]:
def offsetPhases(data):
    for run in data:
        for phase in run:
            delta = phase[0][0]
            for e, dtp in enumerate(phase):
                phase[e][0] -= delta
    return data

In [50]:
def cleanPhases(data):
    for run in data:
        for phase in run:
            last = len(phase)-1
            while last > 0:
                if phase[last][1] == phase[last-1][1]:
                    phase.pop()
                    last -= 1
                else:
                    break
    return data

In [51]:
def undoOffset(data):
    lastEval = list(range(10))
    for phase in range(10):
        maximum = float('-inf')
        for a, alg in enumerate(data):
            maximum = max(maximum, data[a][phase][-1][0])
        lastEval[phase] = maximum
    lastEval = [max(lastEval)] * 10
    lastEval = [sum(x) for x in zip(lastEval, list(range(1,11)))]
    lastEval.insert(0,0)
    delta = np.cumsum(lastEval)
    for run in data:
        for p, phase in enumerate(run):
            for e, _ in enumerate(phase):
                phase[e][0] += delta[p]
            if phase[-1][0] != delta[p+1]-1:
                phase.append([delta[p+1]-1, phase[-1][1]])
    return data

In [52]:
def averagePhase(data):
    uniqueEvals = getUniqueEvals(data)
    avgPhase = []
    for ue in sorted(uniqueEvals):
        obj = verticalCutAt(data, ue).mean()
        avgPhase.append([ue, obj])
    return avgPhase

In [53]:
def averageRunWithPhases(data):
    data = np.array(data)
    avgRun = []
    for phase in range(10):
        avgRun.append(averagePhase(data[:,phase]))
    return avgRun

In [54]:
def normalisePhases(data):
    res = np.copy(data)
    for phase in range(10):
        smallest = float("inf")
        largest = float("-inf")
        for a, alg in enumerate(data):
            smallest = min(smallest, np.array(alg[phase])[:,1].min())
            largest = max(largest, np.array(alg[phase])[:,1].max())
        for a, alg in enumerate(data):
            for i, dtp in enumerate(alg[phase]):
                res[a][phase][i] = [dtp[0], (dtp[1]-smallest)/(largest-smallest)]
    return res

In [55]:
def removePhases(data):
    res = []
    for i, alg in enumerate(data):
        res.append([])
        for phase in alg:
            for dtp in phase:
                res[i].append(dtp)
    return res

In [56]:
def performance(data):
    res = []
    for i, alg in enumerate(data):
        res.append([])
        for phase in alg:
            if len(phase) == 1:
                res[i].append([phase[0][1], phase[0][1]])
            else:
                ph = np.array(phase)
                res[i].append([auc(ph[:,0], ph[:,1])/ph[:,0][-1], ph[:,1][-1]])
    return res

In [67]:
class BlackMagic():
    def __init__(self, data):
         self.data = data

    def plot(self,log=False,xl=None,title=None, col=0):
#         plt.plot([self.data[0][0][0]], [self.data[0][0][1]], marker='o', markersize=8, color="green")
        for j,run in enumerate(self.data):
#             plt.plot([run[1][0]], [run[1][1]], marker='x', markersize=8, color="red")
            x = []
            y = []
            for dtp in run:
                if log:
                    x.append(math.log(dtp[0],10))
                else:
                    x.append(dtp[0])
                y.append(dtp[1])
            plt.step(x, y, color=colours[j],where='post')
        if xl is not None:
            plt.xlim((-50, xl))
#         if title is not None:
#             plt.title(title)
        plt.gca().axes.get_xaxis().set_visible(False)
        plt.gca().get_yaxis().set_visible(False)
        custom_lines = []
        for x in range(len(self.data)):
            custom_lines.append(Line2D([0], [0], color=colours[x], lw=4))
        #plt.gca().legend(custom_lines, labels)
#         plt.show()
        plt.savefig('stepplot_multiMutations/'+title)
    
    def heatmap(self,title=''):
        maxEpoch = []
        for run in self.data:
            maxEpoch.append(run[-1][0])
        maxEpoch = max(maxEpoch)
        X = [x for x in list(range(1, int(maxEpoch+1), 10))]
        Y = list(range(1, len(self.data)+1))
        data = np.zeros((len(X), len(self.data)))
        for i, count in enumerate(X):
            data[i] = verticalCutAt(self.data, count)
        df = pd.DataFrame(np.transpose(data), index=Y, columns=X)
        sns.heatmap(df, cbar=False)
        plt.gca().axes.get_xaxis().set_visible(False)
        plt.gca().get_yaxis().set_visible(False)
#         plt.show()
        plt.savefig('heatmaps_multiMutations/'+title)

colours = ['green','blue','black','red']
# labels = ['Linkern','Insertion','CLK+Insertion']
labels = ["BF", "REA", "PI", "PI+BF"]

## Visualisations

In [2]:
typ = ["items", "cities"]
instances = ["a280_n279_bounded-strongly-corr_01", "a280_n1395_uncorr-similar-weights_05", "a280_n2790_uncorr_10", "fnl4461_n4460_bounded-strongly-corr_01", "fnl4461_n22300_uncorr-similar-weights_05", "fnl4461_n44600_uncorr_10", "pla33810_n33809_bounded-strongly-corr_01", "pla33810_n169045_uncorr-similar-weights_05", "pla33810_n338090_uncorr_10"]
dropPercent = ["0.010000", "0.030000", "0.100000", "0.300000"]

fig= plt.figure(figsize=(10,10))

for t in typ:
    if t == "items":
        optimisation = ["viaBitflip", "viaStructuralDiversity", "viaPackIterative", "viaPackIterativeAndBitflip"]
    else:
        optimisation = ["viaJump", "viaCLK", "viaCLKAndJump"]
    for i in notebook.tqdm(instances):
        for k in dropPercent:
            alldata = []
            for col, o in enumerate(optimisation):
                data = readData("output/instances_"+i+".ttp_"+t+"_"+o+"_"+k+".txt")
                #data = offsetPhases(data)
                data = cleanPhases(data)
                data = averageRunWithPhases(data)
                alldata.append(data)
            alldata = normalisePhases(alldata)
            alldata = undoOffset(alldata)
            alldata = removePhases(alldata)
            bm = BlackMagic(alldata)
            plt.figure()
            bm.plot(title=(i+"_"+t+"_k="+str(int(float(k)*100))), col=col)
            plt.close()
            plt.figure()
            bm.heatmap(title=(i+"_"+t+"_k="+str(int(float(k)*100))))
            plt.close()

SyntaxError: invalid syntax (<ipython-input-2-ba3e74729f1e>, line 14)